# Yelp API - Lab

The previously deployed lab on working around building a GIS with Yelp API and Folium can be found [here](https://github.com/learn-co-curriculum/dsc-2-15-10-yelp-api-gis-lab/tree/a56358c2d0c2daf569a5f50937c4c27463aadb1a) (not relevant for new students).


## Introduction 

Now that we've seen how the Yelp API works, and some basic Folium visualizations its time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [1]:
import requests
api_key='ywXXimItH3Lm2tV-wsoSrayzq7hJWeSYvz20Aigqd-yJ1-QYM9btr4UKf17wI6gGo529szJdVytRctmAivlo-6euxjlwwPwdXrK5NEnR6psT55kJ1hftsBZaz4WSXHYx'
client_id='_-Dcw4-lI3bWd4Io4iIOEQ'
term = 'Pizza'
location = 'NYC'
url='https://api.yelp.com/v3/businesses/search'
headers={'Authorization': 'Bearer {}'.format(api_key)}
params={'terms': term.replace('', '+'), 'location': location.replace('','+')}
response=requests.get(url, headers=headers, params=params)
response

<Response [200]>

In [2]:
response.json().keys()

dict_keys(['businesses', 'total', 'region'])

In [3]:
len(response.json()['businesses'])

20

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.josn().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [4]:
import pandas as pd
import time

def yelp_call(params, api_key):
    url='https://api.yelp.com/v3/businesses/search'
    headers={'Authorization': 'Bearer {}'.format(api_key)}
    response=requests.get(url, headers=headers, params=params)
    df=pd.DataFrame(response.json()['businesses'])
    return df

def all_results(params, api_key):
    num=response.json()['total']
    print('{} total matches found.'.format(num))
    cur=0
    dfs=[]
    while cur<num and cur<1000:
        params['offset']=cur
        dfs.append(yelp_call(params, api_key))
        time.sleep(1)
        cur+=50
    df=pd.concat(dfs, ignore_index=True)
    return df
df=all_results(params, api_key)
print(len(df))

24600 total matches found.
400


In [9]:
df

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,levain-bakery-new-york,"[{'alias': 'bakeries', 'title': 'Bakeries'}]","{'latitude': 40.7799404643263, 'longitude': -7...",(212) 874-6080,8366.018270,H4jJ7XB3CetIr1pg56CczQ,https://s3-media2.fl.yelpcdn.com/bphoto/9uUdfb...,False,"{'address1': '167 W 74th St', 'address2': '', ...",Levain Bakery,+12128746080,NaN,4.5,7398,[],https://www.yelp.com/biz/levain-bakery-new-yor...
1,the-halal-guys-new-york-2,"[{'alias': 'foodstands', 'title': 'Food Stands...","{'latitude': 40.7617559582706, 'longitude': -7...",,6448.222046,xEnNFXtMLDF5kZDxfaCJgA,https://s3-media1.fl.yelpcdn.com/bphoto/24lb-V...,False,"{'address1': 'W 53rd St 6th Ave', 'address2': ...",The Halal Guys,,NaN,4.0,9132,[],https://www.yelp.com/biz/the-halal-guys-new-yo...
2,ippudo-ny-new-york-7,"[{'alias': 'ramen', 'title': 'Ramen'}]","{'latitude': 40.73092, 'longitude': -73.99015}",,2913.733501,44SY464xDHbvOcjDzRbKkQ,https://s3-media1.fl.yelpcdn.com/bphoto/zF3Egq...,False,"{'address1': '65 4th Ave', 'address2': '', 'ad...",Ippudo NY,,NaN,4.0,9694,"[pickup, delivery]",https://www.yelp.com/biz/ippudo-ny-new-york-7?...
3,katzs-delicatessen-new-york,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...","{'latitude': 40.72222, 'longitude': -73.98751}",(212) 254-2246,2192.570387,V7lXZKBDzScDeGB8JmnzSA,https://s3-media1.fl.yelpcdn.com/bphoto/B6xNx7...,False,"{'address1': '205 E Houston St', 'address2': '...",Katz's Delicatessen,+12122542246,NaN,4.0,10859,"[pickup, delivery]",https://www.yelp.com/biz/katzs-delicatessen-ne...
4,the-metropolitan-museum-of-art-new-york-3,"[{'alias': 'artmuseums', 'title': 'Art Museums'}]","{'latitude': 40.779254271604, 'longitude': -73...",(212) 535-7710,8760.066939,jVncyqXwlx_D9f2xZn05tg,https://s3-media2.fl.yelpcdn.com/bphoto/UZ5-dy...,False,"{'address1': '1000 5th Ave', 'address2': '', '...",The Metropolitan Museum of Art,+12125357710,NaN,4.5,2670,[],https://www.yelp.com/biz/the-metropolitan-muse...
5,central-park-new-york,"[{'alias': 'parks', 'title': 'Parks'}]","{'latitude': 40.782418740809, 'longitude': -73...",(212) 310-6600,8999.811105,xOw4aafJRji9XomGunlvcQ,https://s3-media2.fl.yelpcdn.com/bphoto/a_T1R7...,False,"{'address1': '14 E 60th St', 'address2': '', '...",Central Park,+12123106600,NaN,5.0,2284,[],https://www.yelp.com/biz/central-park-new-york...
6,los-tacos-no-1-new-york,"[{'alias': 'tacos', 'title': 'Tacos'}]","{'latitude': 40.7425547, 'longitude': -74.0060...",(212) 256-0343,3972.984422,jnEv25Y2DosTq2sNnvmC9g,https://s3-media2.fl.yelpcdn.com/bphoto/FU44TY...,False,"{'address1': '75 9th Ave', 'address2': '', 'ad...",Los Tacos No.1,+12122560343,NaN,4.5,2612,[],https://www.yelp.com/biz/los-tacos-no-1-new-yo...
7,bibble-and-sip-new-york-2,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...","{'latitude': 40.76282, 'longitude': -73.98518}",(646) 649-5116,6421.465724,WHRHK3S1mQc3PmhwsGRvbw,https://s3-media2.fl.yelpcdn.com/bphoto/b4_G7D...,False,"{'address1': '253 W 51st St', 'address2': '', ...",Bibble & Sip,+16466495116,NaN,4.5,4250,[],https://www.yelp.com/biz/bibble-and-sip-new-yo...
8,burger-and-lobster-new-york-4,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 40.74007, 'longitude': -73.99344}",(646) 833-7532,3797.065266,UA2M9QFZghe-9th2KwLoWQ,https://s3-media3.fl.yelpcdn.com/bphoto/1rHd7F...,False,"{'address1': '39 W 19th St', 'address2': '', '...",Burger & Lobster,+16468337532,NaN,4.0,4932,"[restaurant_reservation, pickup, delivery]",https://www.yelp.com/biz/burger-and-lobster-ne...
9,gramercy-tavern-new-york,"[{'alias': 'newamerican', 'title': 'American (...","{'latitude': 40.738527, 'longitude': -73.988417}",(212) 477-0777,3755.058106,veq1Bl1DW3UWMekZJUsG1Q,https://s3-media4.fl.yelpcdn.com/bphoto/1C20qn...,False,"{'address1': '42 E 20th St', 'address2': '', '...",Gramercy Tavern,+12124770777,NaN,4.5,2721,[],https://www.yelp.com/biz/gramercy-tavern-new-y...


In [9]:
a=[{'Feature1': 'aa1','Feature2': 'bb1','Feature3': 'cc2' },
 {'Feature1': 'aa2','Feature2': 'bb2' },
 {'Feature1': 'aa1','Feature2': 'cc1' }
 ]
b=['num1','num2','num3']


dl= pd.DataFrame({'num':b, 'dic':a })
dl.head()

,num,dic
0,num1,"{'Feature1': 'aa1', 'Feature2': 'bb1', 'Featur..."
1,num2,"{'Feature1': 'aa2', 'Feature2': 'bb2'}"
2,num3,"{'Feature1': 'aa1', 'Feature2': 'cc1'}"


## Exploratory Analysis

Take the restaurants from the previous question and do an intial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [6]:
df.shape

(400, 16)

In [5]:
df.describe()

,distance,rating,review_count
count,400.000000,400.000000,400.000000
mean,4320.586211,4.116250,1216.787500
std,2549.001457,0.310002,1146.320452
min,543.270193,3.500000,188.000000
25%,2474.645420,4.000000,622.750000
50%,3694.990211,4.000000,921.500000
75%,5950.696026,4.500000,1426.500000
max,12300.458685,5.000000,10859.000000


In [5]:
price_dict = {'$':1, '$$':2, '$$$':3, '$$$$':4}
df.price = df.price.map(price_dict)
df.price.fillna(0, inplace=True)
df.price

0      2.0
1      1.0
2      2.0
3      2.0
4      0.0
5      0.0
6      1.0
7      1.0
8      2.0
9      4.0
10     0.0
11     2.0
12     1.0
13     0.0
14     4.0
15     2.0
16     2.0
17     1.0
18     4.0
19     2.0
20     2.0
21     3.0
22     1.0
23     4.0
24     4.0
25     3.0
26     1.0
27     0.0
28     1.0
29     0.0
      ... 
370    3.0
371    2.0
372    2.0
373    2.0
374    2.0
375    2.0
376    2.0
377    1.0
378    2.0
379    2.0
380    1.0
381    2.0
382    2.0
383    3.0
384    2.0
385    2.0
386    2.0
387    2.0
388    2.0
389    1.0
390    2.0
391    2.0
392    3.0
393    1.0
394    1.0
395    2.0
396    2.0
397    3.0
398    2.0
399    3.0
Name: price, Length: 400, dtype: float64

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [6]:
import folium
long=-73.97
lat=40.78
base_map = folium.Map([lat, long], zoom_start=13)
base_map

In [9]:
latitude = [d.get('latitude') for d in df.coordinates]
longitude = [d.get('longitude') for d in df.coordinates]
points = list(zip(latitude, longitude))
for row in df.index:
    lat_long = df['coordinates'][row]
    lat = lat_long['latitude']
    long = lat_long['longitude']
    name = df['name'][row]
    rating = df['rating'][row]
    price = df['price'][row]
    details = '{}\nPrice: {} Rating:{}'.format(name, str(price), str(rating))
    popup = folium.Popup(details, parse_html=True)
    marker=folium.Marker(location=[lat, long], popup=popup)
    marker.add_to(base_map)
base_map

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!